### Imports

In [1]:
import pandas as pd
import numpy as np
import glob
import os
import seaborn as sns
from seaborn import plt
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import RidgeCV

from __future__ import division, unicode_literals
import re
import gensim
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity


%matplotlib inline

/Users/veenakumar/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/veenakumar/anaconda2/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


Combine all **reviews** df's

In [2]:
path = '/Users/veenakumar/ds/metis/metisgh/sf16_ds4/local/Projects/5Kojak/airbnbdata/sanfrancisco/sf_reviews/' 
all_files = glob.glob(os.path.join(path, "*.csv"))

df_from_each_file = (pd.read_csv(f) for f in all_files)
df = pd.concat(df_from_each_file, ignore_index=True)

In [256]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 706464 entries, 0 to 706463
Data columns (total 2 columns):
listing_id    706464 non-null int64
comments      706293 non-null object
dtypes: int64(1), object(1)
memory usage: 10.8+ MB


In [4]:
df.drop(['id','date','reviewer_id','reviewer_name'],axis=1,inplace=True)

In [198]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 706464 entries, 0 to 706463
Data columns (total 2 columns):
listing_id    706464 non-null int64
comments      706293 non-null object
dtypes: int64(1), object(1)
memory usage: 10.8+ MB


In [199]:
df.loc[df['listing_id']==982029]

,listing_id,comments
90720,982029,The location is amazing. The house is a Victo...
90721,982029,We had a great time at this lovely apartment t...
90722,982029,The Pac Heights home was wonderful! Fully stoc...
90723,982029,Amazing location! Amazing apartment! Super cl...
90724,982029,"Great, responsive host. Thanks for everything!..."
90725,982029,We stayed at this lovely apartment fr a week w...
90726,982029,Loved this apartment! It was very spacious and...
90727,982029,Salima's place was perfect for my team's offsi...
90728,982029,"Salima's apartment is absolutely beautiful, lo..."
90729,982029,We thought this was a great option for the pri...


### Cleaning df

Removing duplicate comments, since the data was scraped at different points in time so there maybe some overlap.

In [6]:
rvws_count = df.drop_duplicates('comments',keep='last')

Looking at reviews by count; only want to use listings that have a large number of reviews

In [7]:
rvws_count = rvws_count.groupby('listing_id').count()

In [8]:
rvws_count = rvws_count[rvws_count['comments']>49]

In [9]:
rvws_count.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1397 entries, 958 to 10119316
Data columns (total 1 columns):
comments    1397 non-null int64
dtypes: int64(1)
memory usage: 21.8 KB


In [10]:
rvws_count.reset_index(inplace=True)

Creating a new df with the listing id's that have 50 or more reviews

In [228]:
rvws_count.head()

,listing_id,comments
0,958,52
1,2822,53
2,3850,118
3,5193,241
4,5858,113


In [11]:
reviews = df.loc[df['listing_id'].isin(rvws_count.listing_id)]

In [51]:
# Drop rows with null comments
reviews = reviews.dropna()
reviews = reviews.drop_duplicates('comments',keep='last')

In [56]:
## Convert to Unicode and ignore errors
# reviews["comments"] = reviews.comments.apply(lambda val: unicode(val, errors="ignore"))

In [57]:
reviews.reset_index(inplace=True)

In [233]:
reviews.head()

,listing_id,comments
0,1417398,Catherine and Gabe were away when we arrived. ...
1,1417398,"We had a warm welcome from Catherine, Gabe and..."
2,1417398,Great room. It's nice to share a space but sti...
3,1417398,The room was perfect for our stay with a priva...
4,1417398,Everything was perfect! Catherine and Gabe wer...


In [232]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138936 entries, 0 to 138935
Data columns (total 2 columns):
listing_id    138936 non-null object
comments      138936 non-null object
dtypes: object(2)
memory usage: 2.1+ MB


In [234]:
reviews.to_csv("./reviews_clean_FINAL.csv")

In [231]:
reviews.drop('index',axis=1,inplace=True)

In [60]:
reviews.listing_id = reviews.listing_id.map(str)

### Word2Vec

In [20]:
import re, string
from string import maketrans
from gensim.models import Word2Vec

In [21]:
# Note, to use Google Vecs need to set binary=True in load_word2vec_format()
google_path = '/Users/veenakumar/ds/metis/metisgh/sf16_ds4/local/Projects/5Kojak/GoogleNews-vectors-negative300.bin'

In [22]:
glove_path = '/Users/veenakumar/ds/metis/metisgh/sf16_ds4/local/Projects/5Kojak/glove.6B/glove.6B.300d.txt'

In [23]:
model = gensim.models.Word2Vec.load_word2vec_format(glove_path) 

In [238]:
with open('./model_glove_final.pkl', 'wb') as f:
    pickle.dump(model, f)

In [24]:
model.index2word

[u'the',
 u',',
 u'.',
 u'of',
 u'to',
 u'and',
 u'in',
 u'a',
 u'"',
 u"'s",
 u'for',
 u'-',
 u'that',
 u'on',
 u'is',
 u'was',
 u'said',
 u'with',
 u'he',
 u'as',
 u'it',
 u'by',
 u'at',
 u'(',
 u')',
 u'from',
 u'his',
 u"''",
 u'``',
 u'an',
 u'be',
 u'has',
 u'are',
 u'have',
 u'but',
 u'were',
 u'not',
 u'this',
 u'who',
 u'they',
 u'had',
 u'i',
 u'which',
 u'will',
 u'their',
 u':',
 u'or',
 u'its',
 u'one',
 u'after',
 u'new',
 u'been',
 u'also',
 u'we',
 u'would',
 u'two',
 u'more',
 u"'",
 u'first',
 u'about',
 u'up',
 u'when',
 u'year',
 u'there',
 u'all',
 u'--',
 u'out',
 u'she',
 u'other',
 u'people',
 u"n't",
 u'her',
 u'percent',
 u'than',
 u'over',
 u'into',
 u'last',
 u'some',
 u'government',
 u'time',
 u'$',
 u'you',
 u'years',
 u'if',
 u'no',
 u'world',
 u'can',
 u'three',
 u'do',
 u';',
 u'president',
 u'only',
 u'state',
 u'million',
 u'could',
 u'us',
 u'most',
 u'_',
 u'against',
 u'u.s.',
 u'so',
 u'them',
 u'what',
 u'him',
 u'united',
 u'during',
 u'before',

In [25]:
def get_doc_vecs(model, docs):
    stoplist = stopwords.words('english')
    words_counted = 0
    words_skipped = 0
    docs_vecs = []
    for doc in docs:
        doc_vecs = []
        try:
            regex = re.compile('[%s]' % re.escape(string.punctuation))
            doc = doc.lower().split()
            for word in doc:
                if word not in stoplist:
                    try:
                        word = str(word)
                        word = regex.sub('',word)
                        doc_vecs.append(model[word])
                        words_counted += 1
                    except:
                        words_skipped += 1
                        pass
            docs_vecs.append(np.array(doc_vecs).mean(axis=0))
        except:
            print "Failed Doc!"
            pass
    return docs_vecs

In [61]:
review_vecs = get_doc_vecs(model, reviews.comments)

In [28]:
from gensim.similarities import MatrixSimilarity 

In [62]:
vecs = np.empty((len(review_vecs), 300))
for i in range(len(review_vecs)):
    vecs[i, : ] = review_vecs[i]

In [63]:
review_index = MatrixSimilarity(gensim.matutils.Dense2Corpus(vecs.transpose()))

Pickling model

In [237]:
import pickle

with open('./review_matrix_aggreg_final.pkl', 'wb') as f:
    pickle.dump(review_index, f)

Getting the top scores for the input text

In [240]:
scores = [score for score in review_index[get_doc_vecs(model, ['near lots of bars and public transportation. cozy beds and great for one or two people.'])[0]]]
indexes = range(0,len(review_vecs))
topthousand = sorted(zip(scores, indexes), reverse=True)[:1000]

In [241]:
dfindexes = [tup[1] for tup in topthousand]

In [242]:
dfvectors = [tup[0] for tup in topthousand]

In [244]:
results = reviews.iloc[dfindexes]
# results.drop('index',axis=1,inplace=True)

In [245]:
results.head()

,listing_id,comments
127190,934264,We stayed in Lenore's apartment for two nights...
2715,1910794,"We stayed in the apartement with four people, ..."
23393,1292198,Very well located near public transport to dow...
28330,307891,An absolute jewel. Staying with a friend for ...
107534,450551,Kevin's apartment is perfectly located. There ...


In [246]:
results['vectors'] = dfvectors

/Users/veenakumar/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [218]:
for c in results.comments[:10]:
    print c

Cozy, warm room and close to many good cafes and restaurants.
Great neighborhood.  Loved how there was much to do within walking distance; park, bars, restaurants, and shopping! Room was spacious, clean and beds were comfortable. Thank you again
Great space. Very comfortable bed, full kitchen and nice little eating area. So appreciated the coffee and tea supplied. Lots of restaurants within walking distance. My sister and I felt entirely safe walking at night in the neighborhood. 
Eduardo was a great host. The cottage was very cozy and within walking distance of several nice restaurants and coffee shops. 10/10 would recommend!
Great location, very spacious room with everything you'll need during your time here, and comfy bed and comforter/blankets. The hosts are very attentive and provide you with goodies like water, coffee, beer, wine, toiletries, SF pamphlets, and restaurant menus. They have a very cute cat. Lovely place! 
We had a warm welcome from Kate and her family. The room is q

In [247]:
results_sorted = results.groupby('listing_id').sum().sort('vectors',ascending=False)

/Users/veenakumar/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [248]:
results_sorted.reset_index(inplace=True)

In [250]:
#changing the data type of listing_id to match the data type of 'id' in results_listings table
#so that I can successfully merge them later on.

results_sorted.listing_id = results_sorted.listing_id.map(int)

In [251]:
results_sorted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 2 columns):
listing_id    614 non-null int64
vectors       614 non-null float64
dtypes: float64(1), int64(1)
memory usage: 9.7 KB


In [252]:
results_listings = listings[listings['id'].isin(results_sorted.listing_id.astype(int))]

I want to sort the listings by the 'vectors' column, since a sum means more reviews for that listing i.e. high chance that it is most closely related to text input by user.


I tried to merge results_listings and results_sorted several different ways in order to add the vectors column but I kept getting an empty dataframe! After many such trials I finally looked at ".info" for each df and found that the id and listing_id were of different *data types*!!! Once I changed them to the same data type I was able to successfully merge them. Lesson here is to never take the smallest things for granted; the mistake might appear in the simplest areas!

In [253]:
final = pd.merge(results_listings,results_sorted,left_on='id',right_on='listing_id').sort('vectors',ascending=False)

/Users/veenakumar/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [254]:
final.head()

,name,id,bedrooms,bathrooms,accommodates,description,listing_url,latitude,longitude,neighbourhood_cleansed,...,picture_url,price,property_type,review_scores_rating,summary,thumbnail_url,xl_picture_url,zipcode,listing_id,vectors
511,Classic Russian Hill flat in SF!,734839,2.0,1.0,6,"This charming Russian Hill flat, in an Edwardi...",https://www.airbnb.com/rooms/734839,37.801526,-122.409400,North Beach,...,https://a2.muscache.com/ic/pictures/10382187/a...,330.0,Apartment,98.0,"This charming Russian Hill flat, in an Edwardi...",https://a2.muscache.com/ic/pictures/10382187/a...,https://a2.muscache.com/ic/pictures/10382187/a...,94109.0,734839,6.122040
508,CHARMING NORTH BEACH ALLEY HIDEAWAY,1178260,2.0,1.5,4,CHARMING NORTH BEACH ALLEY HIDEAWAY In the hea...,https://www.airbnb.com/rooms/1178260,37.801543,-122.401358,North Beach,...,https://a2.muscache.com/ic/pictures/17888339/f...,250.0,Apartment,99.0,NaN,https://a2.muscache.com/ic/pictures/17888339/f...,https://a2.muscache.com/ic/pictures/17888339/f...,94133.0,1178260,6.116649
384,Beautiful Upscale House Private BR,38607,1.0,1.0,2,Enjoy one of the absolute best and upscale nei...,https://www.airbnb.com/rooms/38607,37.781975,-122.458092,Presidio Heights,...,https://a1.muscache.com/ic/pictures/844895/8f1...,139.0,House,96.0,NaN,https://a1.muscache.com/ic/pictures/844895/8f1...,https://a1.muscache.com/ic/pictures/844895/8f1...,94118.0,38607,6.115148
249,GORGEOUS FLAT IN GREAT LOCATION,42572,3.0,1.5,6,I am a painter and my work takes up most of my...,https://www.airbnb.com/rooms/42572,37.762218,-122.425424,Mission,...,https://a0.muscache.com/ic/pictures/238885/ba6...,360.0,Apartment,98.0,I am a painter and my work takes up most of my...,https://a0.muscache.com/ic/pictures/238885/ba6...,https://a0.muscache.com/ic/pictures/238885/ba6...,94110.0,42572,5.265322
93,Large central 3BR/2BA with parking,930955,3.0,2.0,7,Our home is located almost right in the middle...,https://www.airbnb.com/rooms/930955,37.759827,-122.448205,Haight Ashbury,...,https://a1.muscache.com/ic/pictures/23505927/9...,500.0,Apartment,92.0,Our home is located almost right in the middle...,https://a1.muscache.com/ic/pictures/23505927/9...,https://a1.muscache.com/ic/pictures/23505927/9...,94114.0,930955,5.257318


In [255]:
for c in final.description[:10]:
    print c


This charming Russian Hill flat, in an Edwardian building built in 1907, is the perfect location for your San Francisco get-away. The building is located on a quiet street in the delightful Russian Hill neighborhood that is full of old world charm. 日本語でのご案内は英語の後にあります。 SAN FRANCISCO SHORT-TERM RESIDENTIAL RENTAL RESISTRATION NUMBER: STR-0000091 Possession of a San FranciscoShort-Term Rental Certificate certifies that the registration certificate holder has agreed to comply with the terms of the San Francisco Short-Term Residential Rental Ordinance (San Francisco Administrative Code Section 41A. This ordinance does not require an inspection of the unit by the City for potential Building, Housing, Fire, or other Code Violations. This charming Russian Hill property, in an Edwardian building built in 1907, is the perfect location for your San Francisco get-away. The building is located on a quiet street in the delightful Russian Hill neighborhood that is full of old world charm, and is conv

Workflow

1. User enters text
2. Vectorize the text --> run it through get_doc_vecs function
3. Matrix similarity and return top50 results
4. Listing table to show top50 listings with info
5. App --> Display top 50 results and on map